# Create tax units from ASEC

Refer to https://github.com/PSLmodels/taxdata/blob/master/cps_data/pycps/pycps.py and https://github.com/ernietedeschi/tcpoverty/blob/master/tcpov2a_make_taxsim27.R but start from scratch.

**Goal: Add a unique tax unit number for each person.**

This tax unit number is the line number of the primary filer, or the minimum line number of the spouses for married filers.

It is created in the following way:
1. 

## Setup

In [1]:
import numpy as np
import pandas as pd
from paramtools.parameters import Parameters
import requests

In [2]:
# TODO: Add usecols once we know which columns are needed.
COLS = ['a_age',  # Age
        'a_famrel',  # Family relationship
        'a_ftpt',  # Full or part time student
        'a_lineno',  # Roster line number
        'phf_seq',  # Pointer to sequence number of own family record
                    # in household (ffpos)
        'a_spouse',  # Spouse's line number.
        'a_maritl',  # Marital status
        'dep_stat',  # Dependent status
        'ptotval',  # 
        'a_exprrp',  # Expanded relationship code
        'filestat',  # Tax filer status (6=nonfiler)
       ]
raw = pd.read_csv('/home/mghenis/MaxGhenis/datarepo/pppub19.csv.gz',
                  usecols=[col.upper() for col in COLS])

## Constants and codes

For 2018 tax year.

In [4]:
class FilingParams(Parameters):
    defaults = requests.get(
        'https://raw.githubusercontent.com/PSLmodels/taxdata/master/cps_data/pycps/filing_rules.json'
    ).json()
#     defaults = Path(CUR_PATH, "filing_rules.json").open("r").read()
    label_to_extend = "year"
    array_first = True


filingparams = FilingParams()

In [5]:
CPS_YEAR = 2018
cps_yr_idx = CPS_YEAR - filingparams.label_grid["year"][0]

In [6]:
filingparams.elderly_age[cps_yr_idx]

65

## Preprocess

In [7]:
cps = raw.copy(deep=True)
cps.columns = cps.columns.str.lower()

Note `p_flag`, `s_flag`, and `d_flag` are instantiated as `False`,
then set to `True` throughout the process upon certain conditions.